# Linear regression 

In [19]:
# create synthetic data and use linreg to compute slope and intercept
X = [1,4,8,11,14,18,20,25];
Y = [6,17,22,35,36,39,46,52];
intercept = linreg(X,Y)[1];
slope = linreg(X,Y)[2];

In [29]:
# plot graph
Pkg.add("Plots")
using Plots
f(x) = intercept+slope*x
plotly() # Choose the Plotly.jl backend for web interactivity
plot(X,Y,seriestype=:scatter,title="linear regression")
plot!(f, 0, 30)

INFO: Package Plots is already installed


# HEADING